In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from datetime import datetime
from time import sleep

def get_article_list(page: int = 1, section_code: str = "S1N1") -> list:
    """
    AI타임즈 뉴스 목록 페이지에서 기사 정보 추출
    """
    base_url = "https://www.aitimes.com/news/articleList.html"
    params = {
        "page": str(page),
        "sc_section_code": section_code,  # AI 관련 섹션 코드
        "view_type": "sm"
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/141.0.0.0 Safari/537.36'
    }

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    articles = []

    # 기사 블록 선택
    for div in soup.select("div.altlist-webzine-content"):
        title_tag = div.select_one("h2.altlist-subject a")
        info_items = div.select("div.altlist-info-item")

        if not title_tag or len(info_items) < 2:
            continue

        title = title_tag.get_text(strip=True)
        url = title_tag["href"]
        reporter = info_items[0].get_text(strip=True)
        date = info_items[1].get_text(strip=True)

        articles.append({
            "title": title,
            "url": url,
            "reporter": reporter,
            "date": date
        })

    return articles


def get_article_content(url: str) -> str:
    """
    개별 기사 URL에서 본문 텍스트 추출
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/141.0.0.0 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        content_div = soup.select_one("div.article-body")  # 본문 부분
        if not content_div:
            content_div = soup.select_one("div#article-view-content-div")

        content = content_div.get_text(separator="\n", strip=True) if content_div else ""
        return content
    except Exception as e:
        print(f"본문 수집 실패 ({url}): {e}")
        return ""


def crawl_ai_news(pages: int = 3):
    """
    여러 페이지에 걸쳐 AI 뉴스 기사 수집
    """
    all_articles = []

    for page in range(1, pages + 1):
        print(f"📄 {page}페이지 수집 중...")
        articles = get_article_list(page)

        for article in articles:
            article["content"] = get_article_content(article["url"])
            all_articles.append(article)
            sleep(1)  # 서버 과부하 방지

    return all_articles


def save_to_json(data: list, filename: str = "aitimes_ai_news.json"):
    """
    결과 데이터를 JSON 파일로 저장
    """
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ {filename} 저장 완료 ({len(data)}개 기사)")


def summarize_articles(data: list):
    """
    날짜별 기사 수 등 간단한 통계 출력
    """
    df = pd.DataFrame(data)
    print("\n📊 날짜별 기사 수:")
    print(df["date"].value_counts())

    print("\n📰 최신 기사:")
    print(df.head(3)[["title", "date", "reporter"]])


if __name__ == "__main__":
    data = crawl_ai_news(pages=2)  # 원하는 페이지 수 조정
    save_to_json(data)
    summarize_articles(data)